In [12]:
import json
import os
from os.path import join, getsize
from os import name
import zipfile

In [15]:
data_r = zipfile.ZipFile("label_Pylabel.zip", "r")
data_r.printdir()

File Name                                             Modified             Size
label_PyLabel/                                 2023-08-25 11:32:56            0
label_PyLabel/bitten/                          2023-08-25 11:31:16            0
label_PyLabel/bitten/ad000046.json             2023-04-12 23:59:06        13522
label_PyLabel/bitten/ad000047.json             2023-04-12 23:59:58         9335
label_PyLabel/bitten/ad000050.json             2023-04-13 00:00:58          152
label_PyLabel/bitten/ad000129.json             2023-04-12 23:58:26        32796
label_PyLabel/bitten/ad000134.json             2023-04-12 23:55:36        15416
label_PyLabel/fire/                            2023-08-25 11:31:16            0
label_PyLabel/fire/ad000135.json               2023-04-13 00:45:48        85154
label_PyLabel/fire/ad000136.json               2023-04-13 00:50:26        78840
label_PyLabel/fire/ad000137.json               2023-04-13 00:55:40        54554
label_PyLabel/fire/ad000138.json        

In [16]:
data_r.extractall()

In [2]:
dir_path = ".\label_PyLabel"
# директория с аннотациями из SAM формата json

In [3]:
for root, dirs, files in os.walk(dir_path):
    # print(root)
    print(dirs)
    # print(files)

['bitten', 'fire', 'folded', 'insects', 'mold', 'spot', 'torn']
[]
[]
[]
[]
[]
[]
[]


In [4]:
# получение путей к каждому файлу json с разметкой
def get_all_paths(dir_path):
    paths = []
    for foldername in os.listdir(dir_path):
        path = os.path.join(dir_path, foldername)
        if os.path.isdir(path):
            for filename in os.listdir(path):
                if "json" in filename:
                    abs_path = os.path.join(path, filename)
                    paths.append(abs_path)
                else:
                    continue
    return paths

In [5]:
# вызов функции по получению всех путей к файлам json 
all_paths = get_all_paths(dir_path)

In [6]:
# print(type(all_paths)) #list
# print(len(all_paths)) #83

In [7]:
all_paths[:5]

['.\\label_PyLabel\\bitten\\ad000046.json',
 '.\\label_PyLabel\\bitten\\ad000047.json',
 '.\\label_PyLabel\\bitten\\ad000050.json',
 '.\\label_PyLabel\\bitten\\ad000129.json',
 '.\\label_PyLabel\\bitten\\ad000134.json']

In [8]:
# форматирование слешей (при необходимости)
def get_path_format(path):
    clear_path = []
    for p in path:
        if '\\' in p:
            res = p.replace('\\', '/')
            clear_path.append(res)
    return clear_path

In [9]:
list_path_format = get_path_format(all_paths) # получение скорректированных путей к файлам разметки

Вызов функции

In [10]:
list_path_format[:5]

['./label_PyLabel/bitten/ad000046.json',
 './label_PyLabel/bitten/ad000047.json',
 './label_PyLabel/bitten/ad000050.json',
 './label_PyLabel/bitten/ad000129.json',
 './label_PyLabel/bitten/ad000134.json']

Открываем файлы json из списка, читаем содержимое, находим название класса и координаты сегмента

In [11]:
# ---- MAIN PyLABEL JSON TO YOLO TXT CONVERTER
for format_path in list_path_format:
    with open(format_path, "r") as f:
        data = json.load(f)
        
        # ---- ОСНОВНЫЕ ПЕРЕМЕННЫЕ ---
        shapes = data['shapes'] # данные с названиями классов и координатами полигона
        height = data['imageHeight'] #данные с высотой изображения
        width = data['imageWidth'] # данные с шириной изображения
        image = data['imagePath'] # название изображения
        category_names = ['bitten', 'fire', 'folded', 'insects', 'mold', 'spot', 'torn'] # названия всех классов повреждений
        
        # ------- ЦИКЛ ПО КОНВЕРТАЦИИ
        # извлекаем сырые данные из файла json в формате "индекс категории" "координаты сегмента: x1 y1 .... xn yn"/n
        coords_list = []

        for segment in shapes:
            label = segment["label"]
            # print(label)
            coords = segment["points"]

            #coords_list = []
            if label in category_names:
                ind = category_names.index(label) # получаем индекс класса повреждения для формата yolo, начиная с 0 и далее

                norm_coords = []
                for x, y in coords:
                    x = x / width # нормализуем координаты по оси x в диапазоне от 0 до 1
                    y = y / height # нормализуем координаты по оси y в диапазоне от 0 до 1
                    norm_coords.append(f'{x:.3} {y:.3}') #запись строки в формат с 3 знаками после точки

                segment_str = f'{ind} {norm_coords}'
                coords_list.append(segment_str)
        # --- ЗАПИСЬ ДАННЫХ В СЫРУЮ СТРОКУ

        yolo_raw = ' '.join(map(str, coords_list))
        
        # --- ОЧИСТКА ДАННЫХ ОТ ЗНАКОВ ПУНКТУАЦИИ И ФОРМАТИРОВАНИЕ
        delimiter = '\n'
        result = ''
        
        # делим строки с переносом по ]
        for line in yolo_raw.split('] '):
            result += line + delimiter
        
        #редактируем форматирование и чистим от лишних знаков пунктуации
        yolo_final = result.replace(",", "").replace("'", "").replace("[", "").replace("]", "")
        
        # --- ПРОПИСЫВАЕМ ПУТИ ДЛЯ ЗАПИСИ РЕЗУЛЬТАТОВ КОНВЕРТАЦИИ
        yolo_txt_paths = ['txtlabel/bitten_txt', 'txtlabel/fire_txt', 'txtlabel/folded_txt', 
                  'txtlabel/insects_txt', 'txtlabel/mold_txt', 'txtlabel/spot_txt', 'txtlabel/torn_txt']
        
        # ---- ЦИКЛ СОРТИРОВКИ ДАННЫХ ПО УСЛОВИЮ ИНДЕКС КАТЕГОРИИ == ИНДЕКСУ ПАПКИ КАТЕГОРИИ
        
        for txt_path in yolo_txt_paths:
            #создаем новый путь, если такового не существует
            if not os.path.exists(txt_path):
                os.makedirs(txt_path)
            # если id категории в строке yolo final совпадает с индексом папки категории
            if yolo_final[0] == str(yolo_txt_paths.index(txt_path)):
                print("Совпадение")
                condition = True
                file_name = image.replace('png', 'txt') #меняем расширение в имени изображения
                file_path = os.path.join(txt_path, file_name) #создаем путь к файлу txt
                if condition:
                    with open(file_path, 'w') as f: #записываем форматированную строку
                        f.write(yolo_final)
            else:
                print('Условие ложно, файл не записан')      
        

Совпадение
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Совпадение
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Совпадение
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Совпадение
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не записан
Условие ложно, файл не зап

In [ ]:
# https://docs.ultralytics.com/datasets/detect/ 